In [85]:
from IPython.core.display import HTML
HTML("\n".join(open("c360_style.css", 'r').readlines()))

# Cross X - Preprocesado del dato

<div style="text-align: right"> 21/06/20201 </div>
<div style="text-align: right"> Asier Dorronsoro </div>
<div style="text-align: right"> https://www.linkedin.com/in/asierdorronsoro/ </div>

El objetivo de este notebook es reducir la dimensionalidad de un dataset con variables desconocidas. En el área del marketing digital, al disponer de plataformas tecnológicas tan potentes, se dispone de información muy granular: direcciones IPs, agentes de navegadores, versiones, acciones por usuario y un largo etcetera. 

Así, con este pipeline consigo descartar variables que me aportan ruido siguiendo una metodología basada en conceptos puramente estadísticos. 

## Configuracion

### Python librerias default

In [86]:
import pandas as pd
import time
import numpy as np
from numpy import std, mean
from datetime import timedelta,datetime,date
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
plt.style.use('ggplot')

### Librerías Scikit-Learn

In [87]:
#!pip install imbalanced-learn

In [7]:
from sklearn import metrics
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score, RandomizedSearchCV, cross_validate, cross_val_predict, train_test_split,learning_curve 
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, accuracy_score, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

## Dataset

In [8]:
df = pd.read_csv('export_bq_raw.csv')
df.head()

,target,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_870,f_871,f_872,f_873,f_874,f_875,f_876,f_877,f_878,f_879
0,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
1,1,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0
2,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0
3,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0
4,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0


In [9]:
df.shape

(96881, 880)

### Reduccion de dimensionalidad

La correlación es una técnica básica y primordial antes de modelar un dataset. La correlación nos permite descartar variables rapidamente ya que si dos variables están altamente correlacionadas, una de las dos recoge la información. 

#### Correlacion

In [11]:
corr_cols = df.columns
corr_cols = [x for x in corr_cols if 'target' not in x]
base_corr = df[corr_cols]

In [12]:
start_time = time.time()
base_corrDF = base_corr.corr()
print("--- %s seconds ---" % (time.time() - start_time))

--- 542.2131614685059 seconds ---


<div class="info">
  <span class="closebtn" onclick=""></span>
  <strong>Inicio!</strong> Me decanto por decidir un threshold de .90 tras hacer varios intentos y analizar en detalle las correlaciones entre variables.table
</div>

In [ ]:
def extra_info_trt(keyword1, keyword2, df, correlation_upper_limit):
    keyword1_cols = [col for col in df.columns if keyword1 == col]
    keyword2_cols = [col for col in df.columns if keyword2 in col]
    assess_cols = keyword1_cols + keyword2_cols
    assess = df[assess_cols]
    assess_corrDF = assess.corr()
    #reshape Corr
    assess_corrDF = assess_corrDF.where(np.triu(np.ones(assess_corrDF.shape)).astype(np.bool))
    assess_corrDF = assess_corrDF.stack().reset_index()
    assess_corrDF.columns = ['col1', 'col2', 'correlation']
    assess_corrDF = assess_corrDF[assess_corrDF.col1 != assess_corrDF.col2]
    problem_corr = assess_corrDF[assess_corrDF.correlation > correlation_upper_limit]
    #problem_corr = assess_corrDF
    problem_corr = problem_corr.sort_values(by='correlation', ascending=False).reset_index(drop=True)
    return problem_corr

In [ ]:
def extra_info(keyword1, keyword2, df, correlation_upper_limit):
    keyword1_cols = [col for col in df.columns if keyword1 in col]
    keyword2_cols = [col for col in df.columns if keyword2 in col]
    assess_cols = keyword1_cols + keyword2_cols
    assess = df[assess_cols]
    assess_corrDF = assess.corr()
    #reshape Corr
    assess_corrDF = assess_corrDF.where(np.triu(np.ones(assess_corrDF.shape)).astype(np.bool))
    assess_corrDF = assess_corrDF.stack().reset_index()
    assess_corrDF.columns = ['col1', 'col2', 'correlation']
    assess_corrDF = assess_corrDF[assess_corrDF.col1 != assess_corrDF.col2]
    problem_corr = assess_corrDF[assess_corrDF.correlation > correlation_upper_limit]
    #problem_corr = assess_corrDF
    problem_corr = problem_corr.sort_values(by='correlation', ascending=False).reset_index(drop=True)
    return problem_corr

In [ ]:
res = extra_info('f_1', 'f_23', targetDF, 0.5)
res

<div class="info">
  <span class="closebtn" onclick=""></span>
  <strong>Fin!</strong>
</div>

In [13]:
correlation_upper_limit = .90 
base_corrDF_ = base_corrDF.where(np.triu(np.ones(base_corrDF.shape)).astype(np.bool))

# Reshape de la matriz a (col1, col2, correlation)
base_corrDF_ = base_corrDF_.stack().reset_index()
base_corrDF_.columns = ['col1', 'col2', 'correlation']

# Eliminar la diagonal (col1 = col2)
base_corrDF_ = base_corrDF_[base_corrDF_.col1 != base_corrDF_.col2]

In [14]:
base_corrDF_.head()

,col1,col2,correlation
1,f_1,f_2,-0.686583
2,f_1,f_3,0.524819
3,f_1,f_4,-0.405383
4,f_1,f_5,0.524912
5,f_1,f_6,-0.398434


In [15]:
base_corrDF_.shape

(385881, 3)

In [46]:
# Identificar correlaciones por encima de un umbral
problem_corr = base_corrDF_[base_corrDF_.correlation > correlation_upper_limit]
problem_corr = problem_corr.sort_values(by='correlation', ascending=False).reset_index(drop=True)

In [47]:
problem_corr.to_csv('high_corr_total.csv')
problem_corr.head()

,col1,col2,correlation
0,f_447,f_476,1.0
1,f_35,f_53,1.0
2,f_280,f_288,1.0
3,f_281,f_289,1.0
4,f_571,f_600,1.0


In [48]:
worstCorr = problem_corr['correlation'].argmax()
worstCorr

C:\ProgramData\Anaconda3\envs\P_GDatabase\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


0

In [49]:
varPopDF = pd.read_csv('counts_raw_bq.csv')
varPopDF.head()

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,...,f_870,f_871,f_872,f_873,f_874,f_875,f_876,f_877,f_878,f_879
0,4283,4283,5545,5556,5546,5558,5709,5728,6084,6095,...,6639,8751,11470,23122,11759,7453,5373,3007,1251,96881


In [50]:
problem_corr.head()

,col1,col2,correlation
0,f_447,f_476,1.0
1,f_35,f_53,1.0
2,f_280,f_288,1.0
3,f_281,f_289,1.0
4,f_571,f_600,1.0


Metodología para eliminar aquellas columnas que menor represantación tiene en casos de correlaciones mayores al umbral. 

In [51]:
columnDF_cleaned = pd.DataFrame(df.columns, columns=['column_name'])
removed_list = []
while len(problem_corr) > 1:
    # Identificar problemas con mayor correlacion
    worstCorr = problem_corr['correlation'].argmax()
    worstRow = problem_corr.iloc[worstCorr]
    col1_pop = varPopDF[worstRow.col1][0]
    col2_pop = varPopDF[worstRow.col2][0]
    if col1_pop > col2_pop:
        removed_list.append({'removed_column': worstRow.col2, 'correlated_with': worstRow.col1, 'correlation': worstRow.correlation})
        # Eliminar col2 de columnDF
        columnDF_cleaned = columnDF_cleaned[columnDF_cleaned.column_name != worstRow.col2]
        # Eliminar todas las instancias de col2
        problem_corr = problem_corr[problem_corr.col1 != worstRow.col2]
        problem_corr = problem_corr[problem_corr.col2 != worstRow.col2]
    else:
        removed_list.append({'removed_column': worstRow.col1, 'correlated_with': worstRow.col2, 'correlation': worstRow.correlation})
        # eliminar col1 de columnDF
        columnDF_cleaned = columnDF_cleaned[columnDF_cleaned.column_name != worstRow.col1]
        # eliminar las instancias de col1
        problem_corr = problem_corr[problem_corr.col1 != worstRow.col1]
        problem_corr = problem_corr[problem_corr.col2 != worstRow.col1]

pd.DataFrame(removed_list)

C:\ProgramData\Anaconda3\envs\P_GDatabase\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


,removed_column,correlated_with,correlation
0,f_447,f_476,1.000000
1,f_280,f_288,1.000000
2,f_281,f_289,1.000000
3,f_571,f_600,1.000000
4,f_446,f_475,1.000000
...,...,...,...
237,f_673,f_426,0.903749
238,f_29,f_523,0.902973
239,f_596,f_601,0.902961
240,f_198,f_194,0.902960


In [52]:
# Cuantas variables hemos eliminado?
print(f"Removed: {len(removed_list)}")

# Cuantas variables nos quedan?
print(f"Remaining: {len(columnDF_cleaned)}")

Removed: 242
Remaining: 638


In [53]:
columnDF = columnDF_cleaned
#del columnDF_cleaned

In [54]:
pd.DataFrame(removed_list).to_csv('high_corr_cols.csv', index=False)

In [55]:
pd.DataFrame(columnDF_cleaned).to_csv('post_corr.csv', index=False)

#### Reglas de asociacion

In [83]:
crosstabDF = pd.read_csv('crosstabDFT.csv')
crosstabDF.head()

,target,users,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_870,f_871,f_872,f_873,f_874,f_875,f_876,f_877,f_878,f_879
0,1,16555,603,603,795,797,849,851,904,905,...,785,1177,2035,4945,2545,1453,981,595,148,16555
1,0,80326,3680,3680,4750,4759,4697,4707,4805,4823,...,5854,7574,9435,18177,9214,6000,4392,2412,1103,80326


In [63]:
variables = [c for c in crosstabDF.columns if c not in ['target']]

Hago uso de las reglas de asociación con el fin de identificar el lift que una variable tiene sobre la variable objetivo. Esta técnica hace uso la lógica probabilística detrás de las reglas de asociación. 

Así, todas aquellas que tengan una variabilidad menor de un 0.03 se descarta ya que se observa que no tiene impacto alguno incluir esta variable en el dataset.

In [64]:
condProbList = []
for v in variables:
    P_A = crosstabDF[crosstabDF.target == 1].consumers.sum() / crosstabDF['consumers'].sum()
    P_B = crosstabDF[v].sum() / crosstabDF['consumers'].sum()
    P_AB = crosstabDF[crosstabDF.target == 1][v].sum() / crosstabDF['consumers'].sum()
    S_A = crosstabDF[crosstabDF.target == 1].consumers.sum()
    S_B = crosstabDF[v].sum()
    S_AB= crosstabDF[crosstabDF.target == 1][v].sum() 

    condProbList.append({'A': 'inTarget', 'B': v,  'S_A': S_A, 'S_B': S_B, 'S_AB':S_AB, 'P_A': P_A, 'P_B': P_B, 'P_AB':P_AB, 'P_A_B': P_A * P_B,  'P_A_given_B': P_AB/P_B, 'P_B_given_A': P_AB/P_A, 'Lift_B': P_AB/(P_B*P_A)})

condProbDF = pd.DataFrame(condProbList)

In [65]:
condProbDF.head()

,A,B,S_A,S_B,S_AB,P_A,P_B,P_AB,P_A_B,P_A_given_B,P_B_given_A,Lift_B
0,inTarget,consumers,16555,96881,16555,0.17088,1.000000,0.170880,0.170880,0.170880,1.000000,1.000000
1,inTarget,f_1,16555,4283,603,0.17088,0.044209,0.006224,0.007554,0.140789,0.036424,0.823908
2,inTarget,f_2,16555,4283,603,0.17088,0.044209,0.006224,0.007554,0.140789,0.036424,0.823908
3,inTarget,f_3,16555,5545,795,0.17088,0.057235,0.008206,0.009780,0.143372,0.048022,0.839025
4,inTarget,f_4,16555,5556,797,0.17088,0.057349,0.008227,0.009800,0.143449,0.048143,0.839471


In [66]:
# Umbral de correlacion
correlation_upper_limit = .90  # Correlacion maxima que permitiremos para la variable independiente.  Una variable será eliminada si supera este limite.

# Umbrales de Probabilidad Condicional
conditionalProbabilityB_upperLimit = .90   # Probabilidad condicional maxima de la variable sobre el target. 
minimumLiftB = .01                         # no permitir variables que generan un lift < 0.01 respecto a 1
independentProbabilityB_lowerLimit = .03  

In [67]:
condProbDF.sort_values('P_B_given_A', ascending=False, inplace=True)

# Aplicación de reglas
condProbDF['violates_P_B_given_A'] = ((abs(condProbDF.P_B_given_A) > conditionalProbabilityB_upperLimit))# Si todos los usuarios estan aqui no aporta valor
condProbDF['violates_P_B'] = ((condProbDF.P_B < independentProbabilityB_lowerLimit))# Si la columna B no tiene suficientes usuarios
condProbDF['violates_Lift_B'] = ((abs(1 - condProbDF.Lift_B) < minimumLiftB)) # Tiene mas probabilidad de ser positivo en la variable objetivo que de no serlo

problemDF = condProbDF[condProbDF.violates_P_B_given_A | condProbDF.violates_P_B | condProbDF.violates_Lift_B]

In [84]:
display(HTML('<H4>Eliminación de variables en base a reglas condiciones de probabilidad</H4>'))
display(problemDF)

,A,B,S_A,S_B,S_AB,P_A,P_B,P_AB,P_A_B,P_A_given_B,P_B_given_A,Lift_B,violates_P_B_given_A,violates_P_B,violates_Lift_B
0,inTarget,consumers,16555,96881,16555,0.17088,1.000000,0.170880,0.170880,0.170880,1.000000,1.000000,True,False,True
864,inTarget,f_864,16555,96876,16555,0.17088,0.999948,0.170880,0.170871,0.170889,1.000000,1.000052,True,False,True
879,inTarget,f_879,16555,96881,16555,0.17088,1.000000,0.170880,0.170880,0.170880,1.000000,1.000000,True,False,True
96,inTarget,f_96,16555,95274,16325,0.17088,0.983413,0.168506,0.168045,0.171348,0.986107,1.002740,True,False,True
100,inTarget,f_100,16555,92595,16189,0.17088,0.955760,0.167102,0.163320,0.174837,0.977892,1.023156,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,inTarget,f_250,16555,314,36,0.17088,0.003241,0.000372,0.000554,0.114650,0.002175,0.670938,False,True,False
251,inTarget,f_251,16555,268,29,0.17088,0.002766,0.000299,0.000473,0.108209,0.001752,0.633246,False,True,False
231,inTarget,f_231,16555,131,28,0.17088,0.001352,0.000289,0.000231,0.213740,0.001691,1.250824,False,True,False
252,inTarget,f_252,16555,176,18,0.17088,0.001817,0.000186,0.000310,0.102273,0.001087,0.598507,False,True,False


In [69]:
problemDF.to_csv('association_rules.csv')

In [70]:
watch_out = ['event_min_retailer', 'mercadona_competitor_distance_ratio', 'distance_to_mercadona', 'target_redeemer', 'tenure_months', 'age', 'male', '00m']
problemDF = problemDF[~ ((problemDF.B.str.contains('|'.join(watch_out))) & (problemDF.violates_P_B_given_A == True))]

In [71]:
problemDF.to_csv('association_rules_rev.csv')

#### Reduccion Varianza

Teniendo en cuenta lo que hace el PCA y la relevancia que tiene la varianza en la ejecución de la misma, opto por dejar a un lado aquellas variables que hasta el momento no hayan sido descartadas y tengan poca varianza.

Para poder realizar este ejercicio, dejo a un lado variables binarias. 

In [72]:
column_groups = pd.read_csv('ColTypes.csv')
column_groups.head()

,ColType,sat_name
0,recency,f_1
1,numeric,f_2
2,recency,f_3
3,numeric,f_4
4,recency,f_5


In [73]:
varianceDF = df.copy()
var_cols = varianceDF.columns
varianceDF = varianceDF[var_cols]
nonbinarycols =  list(column_groups[column_groups['ColType'] != 'binary']['sat_name'])

In [74]:
list1_as_set = set(var_cols)
intersection = list1_as_set.intersection(nonbinarycols)
var_cols = list(intersection)

In [75]:
varianceDF= varianceDF[var_cols]

In [76]:
from sklearn.feature_selection import VarianceThreshold

def rmLowVariance(dfT,cutoff):
    sel_v = VarianceThreshold(threshold=(cutoff * (1 - cutoff)))
    mask=sel_v.fit(dfT).get_support()
    new_feat = [] 
    del_feat=[]
    for bool, feature in zip(mask, dfT.columns):   #### dfT is the numeric variables in df
        if bool:
            new_feat.append(feature)
        else:
            del_feat.append(feature)
    
    dfT=dfT[new_feat]
    return (dfT, new_feat, del_feat)


In [77]:
df1, keep_var, del_var = rmLowVariance(varianceDF,0.8)

In [78]:
del_var_df = pd.DataFrame(del_var, columns =['Removed_Var'])
del_var_df.to_csv('var_del.csv')

#### Dataframe Final 

Tras la aplicación de estas tres reglas, conseguimos reducir el dataset siguiendo una metodología simple, lógica pero efectiva. 

Llego a redurcir el dataset en un 40% del número de variables del dataset original. El resultado de este dataset se utilizará a continuación para desarrollar el benchmark sobre modelos de ML supervisados.

In [88]:
print(df.columns.shape)
print(columnDF.shape)
columnDF_cc = columnDF[~columnDF.column_name.isin(problemDF.B.unique())]
print(columnDF_cc.shape)
columnDF_cc = columnDF_cc[~columnDF_cc.column_name.isin(del_var_df.Removed_Var.unique())]
print(columnDF_cc.shape)

(880,)
(638, 1)
(549, 1)
(520, 1)


In [92]:
df.head()

,target,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_870,f_871,f_872,f_873,f_874,f_875,f_876,f_877,f_878,f_879
0,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
1,1,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0
2,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0
3,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0
4,0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,0.0,33.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0


In [93]:
df_final = df[columnDF_cc['column_name'].unique()]
df_final.head()

,target,f_3,f_5,f_6,f_11,f_12,f_13,f_25,f_28,f_30,...,f_868,f_869,f_870,f_871,f_872,f_873,f_874,f_875,f_876,f_877
0,0,33.0,33.0,0.0,33.0,0.0,33.0,33.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,33.0,33.0,0.0,33.0,0.0,33.0,33.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,33.0,33.0,0.0,33.0,0.0,33.0,33.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,33.0,33.0,0.0,33.0,0.0,33.0,33.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,33.0,33.0,0.0,33.0,0.0,33.0,33.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
df_final.to_csv('modelling_df.csv', index=False)
df_final.head()